In [332]:
import numpy as np
#from numba import cuda
import matplotlib.pyplot as plt
from numpy import random

# Classes #

In [333]:
#parent class for animals
class Animal:
  def __init__ (self,id, hungerRate, birthRate, maxLife, moveSpeed, speedModifier, rowPosition, columnPosition):
    self.id=id
    self.hungerRate=hungerRate
    self.birthRate=birthRate
    self.maxLife=maxLife
    self.moveSpeed=moveSpeed
    self.speedModifier=speedModifier
    self.rowPosition=rowPosition
    self.columnPosition=columnPosition
    
    #adjacent tiles at any given spot
    self.moveOptions = [[-1,-1],[0,-1],[1,-1],[-1,0],[1,0],[-1,1],[0,1],[1,1]]
    

  def die(self):
        self.rowPosition=-1
        self.columnPosition=-1
        print("oof")
    
  def breed(self,animalType,Grid,numRows, numCols,animalArr):
    #moved into one of our own and TotalPopuation < MaxPopulation - 1
    #spawn new animalType in free adjacent cell

    #breed chance
    toBirth = False

    rand = random.random()

    if rand<=self.birthRate:
      toBirth = True

    #spawn child
    if (toBirth):

      moveOptionsCopy = self.moveOptions.copy()
      validMoveOptions = moveOptionsCopy.copy()
      count = 0

      
      #remove invalid moves from validMoveOptions
      for i in moveOptionsCopy:
        xpos = i[0]
        ypos = i[1]
        
        #if the adjacent cell is out of bounds then remove that from option
        if self.rowPosition + xpos <= -1 or self.rowPosition + xpos >= numRows or self.columnPosition + ypos <= -1 or self.columnPosition + ypos >= numCols:
          validMoveOptions.pop(count) 
          continue
        #if the adjacent cell is occupied
        if getattr(Grid[self.rowPosition+xpos][self.columnPosition+ypos],'myAnimal') != None:
          validMoveOptions.pop(count)
          continue
        count += 1
      
      if (len(validMoveOptions)>0):  
        BirthOption = random.randint(len(validMoveOptions))

        print("child will be at")  
        Move = validMoveOptions[BirthOption]

        deltaX = self.rowPosition + Move[0]
        deltaY = self.columnPosition + Move[1]

        print(deltaX)
        print(deltaY)

        if animalType == "Wolf":
          #spawn new wolf
          setattr(Grid[deltaX][deltaY], 'myAnimal', 'Wolf')
          newID = len(animalArr)
          #hungerRate, birthRate, maxLife, moveSpeed, speedModifier
          animalArr = np.append(animalArr,Wolf(newID,self.hungerRate,self.birthRate,self.maxLife,self.moveSpeed,self.speedModifier,deltaX,deltaY))
          returnList = [Grid,animalArr]
          return returnList
        else :
          #spawn new sheep
          setattr(Grid[deltaX][deltaY], 'myAnimal', 'Sheep')
          newID = len(animalArr)
          #hungerRate, birthRate, maxLife, moveSpeed, speedModifier
          animalArr = np.append(animalArr,Sheep(newID,self.hungerRate,self.birthRate,self.maxLife,self.moveSpeed,self.speedModifier,deltaX,deltaY))
          returnList = [Grid,animalArr]
          return returnList


    

    

  def move(self,Grid):
    """TODO: Correctly calculate and return the correct move speed"""
    self.totalMove=self.moveSpeed*self.speedModifier
    print("I moved {self.totalMove} spaces!")

  def __str__(self):
    return f"id:{self.id}, position:({self.rowPosition},{self.columnPosition}), hungerRate:{self.hungerRate}, breedRate:{self.birthRate}, maxLife:{self.maxLife}, moveSpeed:{self.moveSpeed}, speedModifier:{self.speedModifier}"

In [334]:
#class for prey
class Sheep(Animal):
    def __init__ (self, id, hungerRate, birthRate, maxLife, moveSpeed, speedModifier, rowPosition, columnPosition):
        super().__init__(id,hungerRate, birthRate, maxLife, moveSpeed, speedModifier, rowPosition, columnPosition)
    
    def eat(self):
        print("yum!")

In [335]:
#class for predator
class Wolf(Animal):
    def __init__ (self, id, hungerRate, birthRate, maxLife, moveSpeed, speedModifier, rowPosition, columnPosition):
        super().__init__(id,hungerRate, birthRate, maxLife, moveSpeed, speedModifier, rowPosition, columnPosition)
    
    def eat(self):
        print("yummy!")

In [336]:
#class for each grid spot
class Grass:
  def __init__(self, hasFood, growthRate, myAnimal):
    self.hasFood= hasFood
    self.growthRate= growthRate
    self.myAnimal= myAnimal
  
  def __str__(self):
    return f"{self.myAnimal}"

# Serial Simulation #

In [337]:
def Simulation():
    #Initialize Field
    numRows = 5
    numCols = 5
    

    Grid = np.empty((numRows,numCols), dtype= Grass)

    #print(Grid)

    #Initialize Grass in each field

    hasFood = True
    growthRate = 1.0
    myAnimal = None

    for i in range(numRows):
        for j in range(numCols):
            Grid[i][j] = Grass(hasFood,growthRate,myAnimal)

    #Initialize Prey
    preySize = 10

    preyArr = np.empty([preySize], dtype=Sheep)

    hungerRate = 1
    birthRate = 1 
    maxLife = 10
    moveSpeed = 1
    speedModifier = 1

    for i in range(preySize):
        #randomly generate x,y coordinates
        rowPosition = random.randint(numRows)
        columnPosition = random.randint(numCols)
        
        #if that spot already has animal then generate again
        while (getattr(Grid[rowPosition][columnPosition],'myAnimal') != None):
            rowPosition = random.randint(numRows)
            columnPosition = random.randint(numCols)

        #update grid
        setattr(Grid[rowPosition][columnPosition], 'myAnimal', 'Sheep')
        preyArr[i] = Sheep(i,hungerRate, birthRate, maxLife, moveSpeed, speedModifier, rowPosition, columnPosition)
    


    #Initialize Predator
    predatorSize = 5     

    predatorArr = np.empty([predatorSize], dtype=Wolf)

    hungerRate = 1.0
    birthRate = 1.0 
    maxLife = 10
    moveSpeed = 1
    speedModifier = 1.0

    for i in range(predatorSize):
        rowPosition = random.randint(numRows)
        columnPosition = random.randint(numCols)
        
        #if that spot already has animal then generate again
        while (getattr(Grid[rowPosition][columnPosition],'myAnimal') != None):
            rowPosition = random.randint(numRows)
            columnPosition = random.randint(numCols)

        #update grid
        setattr(Grid[rowPosition][columnPosition], 'myAnimal', 'Wolf')
        predatorArr[i] = Wolf(i,hungerRate, birthRate, maxLife, moveSpeed, speedModifier, rowPosition, columnPosition)

    # initialize populations

    MaxPopulation = numRows * numCols
    TotalPopulation = predatorSize + preySize

    for i in range(numRows):
        print("\n")
        for j in range(numCols):
              print(Grid[i][j],end= " "),


    print("\n")
    print("we picked predator @")
    testx = getattr(predatorArr[2],'rowPosition')
    testy =getattr(predatorArr[2],'columnPosition')
    print(testx)
    print(testy)
    resultlist = predatorArr[2].breed('Wolf',Grid,numRows, numCols,predatorArr)
    Grid = resultlist[0]
    predatorArr = resultlist[1]         

    for i in range(numRows):
        print("\n")
        for j in range(numCols):
              print(Grid[i][j],end= " "),




  



        

In [339]:
Simulation()



Wolf Sheep Sheep None None 

Sheep Sheep None None None 

Wolf Wolf None Sheep None 

Sheep None None Wolf Sheep 

Sheep Wolf None Sheep Sheep 

we picked predator @
2
0
child will be at
3
1


Wolf Sheep Sheep None None 

Sheep Sheep None None None 

Wolf Wolf None Sheep None 

Sheep Wolf None Wolf Sheep 

Sheep Wolf None Sheep Sheep 

# CUDA Kernel #

In [ ]:
@cuda.jit
def Prey_Predator(grid_size, prey_size, predator_size):
    # Thread id in a 1D block
    # Simulation ID
    id = cuda.threadIdx.x
    
    # # Block id in a 1D grid
    # ty = cuda.blockIdx.x
    # # Block width, i.e. number of threads per block
    # bw = cuda.blockDim.x
    # # Compute flattened index inside the array
    # pos = tx + ty * bw



    # #shared array for results
    # prey_results = cuda.shared.array(num_simulations, int32)
    # predator_results = cuda.shared.array(num_simulations, int32)




    #after simulation
    dprey_results[id] = prey.population
    dpredator_results[id] = predator.population

    
    

In [ ]:
# only using 1 block but many threads
# number of threads
num_simulations = 32
num_block = 1

prey_results = np.empty([num_simulations],dtype= int)
predator_results = np.empty([num_simulations],dtype= int)

#copy to device
dprey_results = cuda.to_device(prey_results)
dpredator_results = cuda.to_device(predator_results)


#kernel call

grid_size = 10
prey_size = 1
predator_size = 2


Prey_Predator[num_block, num_simulations](grid_size, prey_size, predator_size)

#get results to host
prey_results = dprey_results.copy_to_host()
predator_results = dpredator_results.copy_to_host()

In [ ]:
#plotting things


fig = plt.figure()
x = np.linspace(0, 2 * np.pi, 50)
y_sin = np.sin(x)
y_cos = np.cos(x)
plt.title('sin(x) & cos(x)')
plt.xlabel('X axis')
plt.ylabel('Y axis')
plt.plot(x, y_sin, color = "red", linewidth = 1.5, linestyle = "-.", label = "y_sin")
plt.plot(x, y_cos, marker = '+', linestyle = '-', label = 'y_cos')
plt.legend(loc = "upper left")
plt.show()